# Data Visualization

What could be a good dataset to try? How about historical price of Bitcoin?

Data is downloaded from the following source(s):
 - Market capitalization https://www.blockchain.com/charts/market-cap
 - Total circulating bitcoin https://www.blockchain.com/charts/total-bitcoins

In [1]:
import os
import csv
import datetime
import numpy as np
import pandas as pd
# import importlib
import matplotlib
import matplotlib.pyplot
import matplotlib.patheffects

from sklearn.linear_model import LinearRegression, RANSACRegressor

import data
import style
style.use_dark_theme()
# style.use_light_theme()

ModuleNotFoundError: No module named 'numpy'

In [ ]:
def v2str(v):
    d = int(np.maximum(-np.log10(v), 0))
    return ('{{:,.{:d}f}}'.format(d)).format(v)

xfmt = matplotlib.ticker.FuncFormatter(lambda x, pos: v2str(x))
yfmt = matplotlib.ticker.FuncFormatter(lambda y, pos: '$' + v2str(y))

years = matplotlib.dates.YearLocator()             # every year
months = matplotlib.dates.MonthLocator()           # every month
years_fmt = matplotlib.dates.DateFormatter('%Y')

In [ ]:
# data = importlib.reload(data)

In [ ]:
df = data.read(rss='M')
# df = data.read()
df.head(5)

d = df.index                       # Date
s = df['Stock'].values             # Stock
f = df['Norm Mean Flow'].values    # Normalized Mean Flow
f2 = df['Norm Tab Flow'].values    # Normalized Flow from table entries (matches better with PlanB's monthly data)
mc = df['Market Cap'].values       # Market Capitalization (USD)

s2f = s / f               # Stock-to-Flow Ratio
s2f2 = s / f2             # Stock-to-Flow Ratio from table entries

In [ ]:
df.tail(4)

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(9, 4.5), dpi=144)
ax = matplotlib.pyplot.axes([0.06, 0.08, 0.92, 0.82])
ax.semilogy(d, mc / s, label='Price')
ax.semilogy(d, s, label='Count')
ax.semilogy(d, s2f, label='S2F')
ax.semilogy(d, s2f2, label='S2F-2')
ax.legend()
ax.grid()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)
ax.set_ylabel('Price / Tranactions / Count / S2F')
title_text = ax.set_title('Time History', fontweight='bold', fontsize=16)
title_text.set_path_effects([
    matplotlib.patheffects.Stroke(linewidth=1, foreground=(0, 0, 0, 0.7)),
    matplotlib.patheffects.Normal()
])

In [ ]:
np.mean(s2f2 / s2f)

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(9, 4.5), dpi=144)
ax = matplotlib.pyplot.axes([0.06, 0.08, 0.92, 0.82])
ax.plot(d, s2f, label='S2F')
ax.plot(d, s2f2, markersize=2, label='S2F2')
ax.legend()
ax.grid()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)
ax.set_ylabel('Price / Tranactions / Stock / S2F')
title_text = ax.set_title('Time History', fontweight='bold', fontsize=16)
title_text.set_path_effects([
    matplotlib.patheffects.Stroke(linewidth=1, foreground=(0, 0, 0, 0.7)),
    matplotlib.patheffects.Normal()
])

In [ ]:
hh = [
    0,
    df.index.get_loc(pd.to_datetime('2012-11-28'), method='nearest'),
    df.index.get_loc(pd.to_datetime('2016-07-09'), method='nearest'),
    df.index.get_loc(pd.to_datetime('2020-05-11'), method='nearest'),
]
print(hh, np.diff(hh))

In [ ]:
fig = matplotlib.pyplot.figure()
ax = matplotlib.pyplot.axes([0.25, 0.12, 0.72, 0.76])
ax.plot(58.3, 10.08e12, '.', markersize=20, color='#C29E29', label='Gold (SF58.3, 10.08T)')
ax.plot(33.3, 561e9, '.', markersize=20, color='#999999', label='Silver (SF33.3, 561B)')

for i in range(len(hh)):
    b = hh[i]
    e = len(d) if i == len(hh) - 1 else hh[i + 1]
    x = s2f2[b:e]
    #y = p[b:e]
    y = mc[b:e]
    label = 'Genesis' if i == 0 else 'Halving {}'.format(i)
    ax.loglog(x, y, '.', markersize=3, label=label)

ax.legend(loc=2)
ax.grid()
ax.set_xlim((0.1, 250))
ax.set_ylim((1e4, 100e12))

loc = []
for i in range(3, 15):
    loc.append(10 ** i)
ax.yaxis.set_major_locator(matplotlib.ticker.FixedLocator(loc))
ax.xaxis.set_major_formatter(xfmt)
ax.yaxis.set_major_formatter(yfmt)
ax.set_xlabel('S2F')
ax.text(0.8, 2e4, 'Two pizzas for 10k BTC', fontsize=8)
title_text = ax.set_title('Market Value', fontweight='bold', fontsize=16)
title_text.set_path_effects([
    matplotlib.patheffects.Stroke(linewidth=1, foreground=(0, 0, 0, 0.7)),
    matplotlib.patheffects.Normal()
])

In [ ]:
# fig.savefig(os.path.expanduser('~/Downloads/s2f.png'), facecolor='k')

### Data Fitting

In [ ]:
ii = np.sum(mc == 0)
# ix = np.expand_dims(np.log10(s2f[ii:]), 1)
# ix = np.expand_dims(np.log10(s2f2[ii:]), 1)
# iy = np.log10(mc[ii:])

# Up to month 135
ix = np.expand_dims(np.log10(s2f2[ii:136]), 1)
iy = np.log10(mc[ii:136])

linreg = LinearRegression().fit(ix, iy)
ransac = RANSACRegressor().fit(ix, iy)

mx = np.expand_dims(np.logspace(-1, 2.5), 1)
my = 10 ** linreg.predict(np.log10(mx))
print('Linear Regression: log10(y) = {:.4f} * log10(S2F) + {:.4f}'.format(linreg.coef_[0], linreg.intercept_))

my2 = 10 ** ransac.predict(np.log10(mx))
print('RANSAC Regression: log10(y) = {:.4f} * log10(S2F) + {:.4f}'.format(ransac.estimator_.coef_[0], ransac.estimator_.intercept_))

### Comparison to Silver and Gold

S2F and market cap numbers are from:

https://medium.com/@100trillionUSD/bitcoin-stock-to-flow-cross-asset-model-50d260feed12

In [ ]:
fig = matplotlib.pyplot.figure()
ax = matplotlib.pyplot.axes([0.22, 0.12, 0.73, 0.76])
ax.tick_params(axis='y')
# ax.plot(mx, my, '--', linewidth=0.5, color='#666666')
ax.plot(mx, my2, '-.', linewidth=0.5, color='#FF66DD', zorder=-1)
ax.set_xlim((0.1, 250))
ax.set_ylim((1e4, 100e12))
ax.set_xscale('log')
ax.set_yscale('log')
ax.grid()

# From PlanB's article published on 3/22/2019
# ax.plot(62, 8.5e12, '.', markersize=20, color='#C29E29', label='Gold (SF62, 8.5T)')
# ax.plot(22, 308e9, '.', markersize=20, color='#999999', label='Silver (SF22, 308B)')
# ax.text(51, 6e12, 'Gold (SF62, 8.5T)', fontsize=8, ha='right')
# ax.text(17, 2e11, 'Silver (SF22, 308B)', fontsize=8, ha='right')

# From PlanB's article published on 4/27/2020
ax.plot(58.3, 10.08e12, '.', markersize=20, color='#C29E29', label='Gold (SF58.3, 10.08T)')
ax.plot(33.3, 561e9, '.', markersize=20, color='#999999', label='Silver (SF33.3, 561B)')
ax.text(48, 8e12, 'Gold (SF58.3, 10.08T)', fontsize=8, ha='right')
ax.text(27, 5e11, 'Silver (SF33.3, 561B)', fontsize=8, ha='right')

for i in range(len(hh)):
    b = hh[i]
    e = len(d) if i == len(hh) - 1 else hh[i + 1]
    x = s2f2[b:e]
    y = mc[b:e]
    w = np.array(df.index[b:e] - df.index[b], dtype=np.float) / 86400e9 / 365.25 * 12
    label = 'Genesis' if i == 0 else 'Halving {}'.format(i)
    hs = ax.scatter(x, y, c=w, vmin=0, vmax=48, cmap='rainbow_r', s=3, label=label)
loc = []
for i in range(3, 15):
    loc.append(10 ** i)
ax.yaxis.set_major_locator(matplotlib.ticker.FixedLocator(loc))
ax.xaxis.set_major_formatter(xfmt)
ax.yaxis.set_major_formatter(yfmt)
ax.set_axisbelow(True)
ax.set_xlabel('S2F')

cax = fig.add_axes((0.51, 0.2, 0.4, 0.028))
fig.colorbar(hs, cax=cax, orientation='horizontal')
loc = []
for i in range(0, 49, 6):
    loc.append(i)
cax.xaxis.set_major_locator(matplotlib.ticker.FixedLocator(loc))
cax.set_title('Months After Halving')

for i, p in enumerate(((2, 1e6), (10, 1e8), (25, 2e9), (55, 5e10))):
    label = 'Genesis' if i == 0 else 'Halving {}'.format(i)
    ax.text(p[0], p[1], label, fontsize=8)
    
ax.text(s2f2[-1], mc[-1], '  {:,.0f}B'.format(1.0e-9 * mc[-1]), fontsize=5, ha='left', va='baseline')

title_text = ax.set_title('Market Value', fontweight='bold', fontsize=16)
title_text.set_path_effects([
    matplotlib.patheffects.Stroke(linewidth=1, foreground=(0, 0, 0, 0.7)),
    matplotlib.patheffects.Normal()
])

In [ ]:
fig.savefig(os.path.expanduser('~/Downloads/s2f-m2.png'), facecolor='k', dpi=320)

### Price Forecast given specific S2F values

From output above, something like these (may change when data is updated):
```
Linear Regression: log10(y) = 3.3186 * log10(S2F) + 6.2018
RANSAC Regression: log10(y) = 3.2226 * log10(S2F) + 6.3039
```

In [ ]:
# Expect SF = 56 during 2020-2024
# Double to SF = 112 during 2024-2084

for x in (56, 112):
    print('SF = {}'.format(x))
    y = 10 ** linreg.predict(np.log10(np.expand_dims(np.array([x, ]), 0)))[0]
    p = y / df['Stock'][-1]
    print('         Linear: MC = {:9,.2f}B, Price = ${:,.2f}'.format(1e-9 * y, p))
    y = 10 ** ransac.predict(np.log10(np.expand_dims(np.array([x, ]), 0)))[0]
    p = y / df['Stock'][-1]
    print('         RANSAC: MC = {:9,.2f}B, Price = ${:,.2f}'.format(1e-9 * y, p))